# Collecting Tweets for Analysis via Tweepy and the Twitter API

Pulling Twitter data requires a configuring access via Twitter's Developers partnerships. We will use our API access to pull the 3200 most recent tweets from a user and insert them into a MongoDB database.

Will need to add your keys and tokens


In [1]:
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Below utility function will collect all the most recent "tweet_count" tweets for a given twitter handle ("@...."). Because only 200 tweets can be accessed in a given request, function will send multiple requests until tweet_count is reached

In [2]:
# Assumes open twitter API called api
# Returns json
def get_tweets(handle,tweet_count):
    if(tweet_count<=200):
        tw=api.user_timeline(screen_name = handle,count=tweet_count)
        tw_json=[e._json for e in tw]
        return (tw_json)

    tw_json=[]
    cur_max=999999999999999999
    loop_count=tweet_count-tweet_count%200
    for i in range(0,loop_count,200):
         tw=(api.user_timeline(screen_name = handle,count=200,max_id=cur_max))
         tw_json=tw_json+([e._json for e in tw])
         cur_max=tw_json[-1:][0]['id']
    tw=(api.user_timeline(screen_name = handle,count=tweet_count-loop_count,max_id=cur_max))
    tw_json=tw_json+([e._json for e in tw])
    return (tw_json)

In [4]:
tw=get_tweets('@RealDonaldTrump',3200)

Connect to pymongo and create a new collection 'tweets' in the 'twitter' database

In [6]:
import pymongo
from pymongo import MongoClient
c=MongoClient()
tweets=c.twitter.tweets

Insert our Twitter API json into tweets MongoDB

In [13]:
tweets.insert_many(tw)
tweets.find_one()

{'_id': ObjectId('595a619f6569d21c90ce2c18'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Mon Jul 03 14:00:21 +0000 2017',
 'entities': {'hashtags': [{'indices': [22, 34], 'text': 'CharlieGard'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'favorite_count': 27279,
 'favorited': False,
 'geo': None,
 'id': 881875263700783104,
 'id_str': '881875263700783104',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 9279,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': 'If we can help little #CharlieGard, as per our friends in the U.K. and the Pope, we would be delighted to do so.',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Wed Mar 18 13:46:38 +0000 2009',
  'defa